Question 3: Find the closest bus and tram stops at the start and finish of the swimming route. How
many people can be transported within an hour.

Question 4:Can you find which bus and tram lines these are, and can you find their routes?

As the bus and tram stops and therefore the lines had to be identified to answer question 3, question 4 will thus be answered in the process.

In [3]:
import requests
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox
from geopy.distance import great_circle


In [4]:
# Set x,y coordinates for the starting point of the swim
x_coor = 52.372870
y_coor = 4.914240
starting_point = (x_coor,y_coor)

# Apply tags to filter for tram stops
tags = {'public_transport':['stop_position']}

# Obtain features (tram stop locations) from the starting point within a distance of 300
features = ox.features.features_from_point(starting_point, tags = tags, dist = 300)

# Pull tram data from online with requests, using data from maps.amsterdam
tram_data = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=TRAMMETRO_PUNTEN_2022&THEMA=trammetro"
pulled_data = requests.get(tram_data)

# Transform pulled data into GeoDataFrame
gdf = gpd.read_file(pulled_data.text)

# Apply great_circle function from OSMNX and distance function within gdf to select only tram stops in radius
gdf['distance'] = gdf['geometry'].apply(lambda stop: great_circle((x_coor, y_coor), (stop.y, stop.x)).meters)
closest_stops = gdf.nsmallest(5, 'distance')

# Print an overview of tram line/stops data
print(closest_stops)

                     Naam Modaliteit Lijn Lijn_select  RADIUS  \
121  Muziekgebouw Bimhuis       Tram   26          26       5   
122    Kattenburgerstraat       Tram   26          26       5   
76                  Artis       Tram   14          14       5   
79        Mr. Visserplein       Tram   14          14       5   
223    Plantage Lepellaan       Tram   14          14       5   

                         Label                  geometry    distance  
121  26 - Muziekgebouw Bimhuis  POINT (4.91302 52.37725)  493.685432  
122    26 - Kattenburgerstraat  POINT (4.92158 52.37612)  615.546946  
76                  14 - Artis  POINT (4.91117 52.36663)  724.698979  
79        14 - Mr. Visserplein  POINT (4.90570 52.36782)  807.082571  
223    14 - Plantage Lepellaan  POINT (4.91548 52.36513)  864.692573  


For the starting point of the swim there are 2 tramlines servicing the nearest stops:

Formatted as: Line, Frequancy, Stops, Destinations

Line 14, 6/hour, 3 stops, Amsterdam Central Station & Flevopark

Line 26, 8/hour, 2 stops, Amsterdam Central Station & IJburg

Data on the bus stops and lines couldn't be retrieved from OSMNX as trying to filter for nodes tagged highway: "bus_stop" or public_transport: "platform", which are the tags OSM uses for bus stops didn't yield any results. So nearby bus stops where identified manually using OSM.

For the buslines there are 2 lines, but 1 is a night bus and is thus excluded:

Line 43, 4/hour, 2 stops, Amsterdam Central Station & Borneo IJland

The frequancy data is taken from the schedule per line from reisinfo.gvb.nl 

Sources: https://reisinfo.gvb.nl/nl/lijnen/14/dienstregeling/heen, https://reisinfo.gvb.nl/nl/lijnen/26/dienstregeling/heen, https://reisinfo.gvb.nl/nl/lijnen/43/dienstregeling/heen

Based on data from over.gvb.nl on the sizes of the busses each bus has space for 150 passengers (45 seated, 105 standing), while the tram size varies with the smallest and largest being 128 (52 seated, 76 standing) and 145 (50 seated, 125 standing) respectively.

Sources: https://over.gvb.nl/ov-in-amsterdam/voer-en-vaartuigen/tram-in-cijfers/, https://over.gvb.nl/ov-in-amsterdam/voer-en-vaartuigen/bus-in-cijfers/


In [54]:
# Calculate the number of passengers per bus then per hour
passengers_seated = 45
passengers_standing = 105

total_passengers = passengers_seated + passengers_standing

bus_passengers_per_hour = total_passengers * 4
print("Bus passengers per hour:", + bus_passengers_per_hour)

# Calculate the number of passengers per full tram then per hour
# Number of total potential passengers varies per type of tram so we calculate a minimum and maximum,
# based on the trams with the smallest and largest number of passengers possible

# First input the tram minimum passenger data for when the tram would be full
passangers_tram_seated_min = 52
passangers_tram_standing_min = 76

# Then input the tram maximum passenger data for when the tram would be full
passangers_tram_seated_max = 50 #might seem wierd but the tram with the highest total passengers only has 50 seats with 25 to 30 more standing spots
passangers_tram_standing_max = 125

# Now we calculate the minimum and maximum number of people per full tram
passengers_tram_min = passangers_tram_seated_min + passangers_tram_standing_min
passengers_tram_max = passangers_tram_seated_max + passangers_tram_standing_max

# Make a dictionary to map tram names with the frequancy data
tram_data = {"Tram_14": 6 , "Tram_26": 8}

# Define a new function consisting of a while loop to calculate the number of passengers per hour per line
def min_passengers_per_hour(tram_data, min_passengers):
    results = []  # Initialize an empty list to store results
    for tram_name, i in tram_data.items():
        tram_passengers_per_hour_min = i * min_passengers
        print(f"{tram_name}: Minimum passengers per hour per direction = {tram_passengers_per_hour_min}")
        results.append(tram_passengers_per_hour_min)  # Append each result to the list
    return results

# Call the function and pass the dictionary of tram names and frequencies
min_results = min_passengers_per_hour(tram_data, passengers_tram_min)

# Calculate the sum of the minimum passenger results
total_min_passengers = sum(min_results)
print(f"Total minimum passengers per hour for tram: {total_min_passengers}")

# Calculate the total for the situation with the largest trams, multiply by two to account for the two directions the trams and busses go in
print((2*(total_min_passengers + bus_passengers_per_hour)), "people can be transported per hour at minimum")

# Now define a similar function for the maximum number of passangers per hour per line
def max_passengers_per_hour(tram_data, max_passengers):
    results = []  # Initialize an empty list to store results
    for tram_name, i in tram_data.items():
        tram_passengers_per_hour_max = i * max_passengers
        print(f"{tram_name}: Minimum passengers per hour per direction = {tram_passengers_per_hour_max}")
        results.append(tram_passengers_per_hour_max)  # Append each result to the list
    return results

# Call the function and pass the dictionary of tram names and frequencies
max_results = max_passengers_per_hour(tram_data, passengers_tram_max)

# Calculate the sum of the minimum passenger results
total_max_passengers = sum(max_results)
print(f"Total maximum passengers per hour for tram: {total_max_passengers}")

# Calculate the total for the situation with the largest trams, multiply by two to account for the two directions the trams and busses go in
print((2*(total_max_passengers + bus_passengers_per_hour)), "people can be transported per hour at maximum")

Bus passengers per hour: 600
Tram_14: Minimum passengers per hour per direction = 768
Tram_26: Minimum passengers per hour per direction = 1024
Total minimum passengers per hour for tram: 1792
4784 people can be transported per hour at minimum
Tram_14: Minimum passengers per hour per direction = 1050
Tram_26: Minimum passengers per hour per direction = 1400
Total maximum passengers per hour for tram: 2450
6100 people can be transported per hour at maximum


By combining the data on the maximum number of passengers for a full bus and full trams (keeping in mind the the range of maximum passengers for the smallest and largest tram sizes) we get the results as shown above.

To summarize:
Between 4784 and 6100 people can be transported per hour depending on the variable size of the trams.

In [5]:
# Import folium and CircleMarker to show the found tram stops on a map
import folium
from folium import CircleMarker

In [7]:
# Create a Folium map centered at the starting point
m = folium.Map(location = [x_coor, y_coor], zoom_start = 15)

# Add a marker for the starting location to the map
folium.Marker(location=[x_coor, y_coor], popup="Starting Point").add_to(m)

# Add markers for the closest stops
for index, row in closest_stops.iterrows():
    coords = [row['geometry'].y, row['geometry'].x]
    
    # Create a custom icon to match the CircleMarker appearance
    custom_icon = folium.DivIcon(
        html=f'<div style="background-color: red; border-radius: 50%; width: 20px; height: 20px;"></div>',
        icon_size=(10, 10))
    folium.Marker(location=coords, icon=custom_icon).add_to(m)

# Make a list of the locations and their attributes (latitude, longitude), 
# as we didn't manage to get a working code to search for nearest bus stops with OSMNX the stops were added mannualy through their coordinates
locations1 = [
    (52.3727832, 4.9183660),
    (52.3710067, 4.9133407),
    (52.3707704, 4.9160979)]

# Add markers for additional locations
for location in locations1:
    lat, lon = location
    
    # Create a custom icon for additional locations
    custom_icon = folium.DivIcon(
        html=f'<div style = "background-color: blue; border-radius: 50%; width: 20px; height: 20px;"></div>',
        icon_size = (10, 10))
    folium.Marker(location = [lat, lon], icon = custom_icon).add_to(m)

# Print the map with the locations
m

In the above map the starting point of the swim is shown as the blue/white marker. With the nearest bus and tram stops shown in blue en red respectively.

Now comes the answer for question 3 for the end point/finish of the swim

In [8]:
# Set x,y coordinates for the end point/finish of the swim
x_coor2 = 52.374219
y_coor2 = 4.885003
end_point = (x_coor2,y_coor2)

# Apply tags to filter for tram stops
tags = {'public_transport': ['stop_position']}

# Obtain features (tram stop locations) from the starting point within a distance of 300
features2 = ox.features.features_from_point(end_point, tags = tags, dist = 300)

# Pull tram data from online with requests, using data from maps.amsterdam
url = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=TRAMMETRO_PUNTEN_2022&THEMA=trammetro"
response = requests.get(url)

# Transform pulled data into GeoDataFrame
gdf = gpd.read_file(response.text)

# Apply great_circle function from OSMNX and distance function within gdf to select only tram stops in radius
gdf['distance'] = gdf['geometry'].apply(lambda stop: great_circle((x_coor2, y_coor2), (stop.y, stop.x)).meters)
closest_stops = gdf.nsmallest(5, 'distance')

# Print an overview of tram line/stops data
print(closest_stops)

                   Naam Modaliteit              Lijn  Lijn_select  RADIUS  \
205         Westermarkt       Tram           13 | 17        13|17       5   
64   Dam / Paleisstraat       Tram  2 | 12 | 13 | 17  02|12|13|17       5   
130         Marnixplein       Tram                 5            5       5   
215         Bloemgracht       Tram                 5            5       5   
0      Nieuwezijds Kolk       Tram  2 | 12 | 13 | 17  02|12|13|17       5   

                                     Label                  geometry  \
205                  13 | 17 - Westermarkt  POINT (4.88411 52.37403)   
64   2 | 12 | 13 | 17 - Dam / Paleisstraat  POINT (4.89051 52.37316)   
130                        5 - Marnixplein  POINT (4.87847 52.37748)   
215                        5 - Bloemgracht  POINT (4.87645 52.37434)   
0      2 | 12 | 13 | 17 - Nieuwezijds Kolk  POINT (4.89334 52.37607)   

       distance  
205   64.238127  
64   392.049140  
130  572.893757  
215  580.732338  
0    602.50145

So there are 5 tramlines that service the nearest tramstops.

Formatted as: Line, Frequancy, Stops, Destinations

Line 2, 8/hour, 2 stops, 

Line 5, 5/hour, 2 stops, 

Line 12, 7/hour, 2 stops, 

Line 13, 8/hour, 3 stops, 

Line 17, 9/hour, 3 stops, 
              
The frequancy data is taken from the schedule per line from reisinfo.gvb.nl

Sources: https://reisinfo.gvb.nl/nl/lijnen/2/dienstregeling/heen, https://reisinfo.gvb.nl/nl/lijnen/5/dienstregeling/heen, https://reisinfo.gvb.nl/nl/lijnen/12/dienstregeling/, https://reisinfo.gvb.nl/nl/lijnen/13/dienstregeling/heenheen, https://reisinfo.gvb.nl/nl/lijnen/17/dienstregeling/heen

Here we didn't include the buslines as the nearby bus stops are only serviced by the night busses, thus they don't contribute to the ability of participants/viewers to reach the location of the finishline.                    

In [56]:
# Calculating the number of passengers per full tram then per hour
# Number of total potential passengers varies per type of tram so we calculate a minimum and maximum,
# based on the trams with the smallest and largest number of passengers possible
#Here we are using the tram, min, max -passenger data for when a tram is full from before

# Make a dictionary to map tram names with the frequancy data
tram_data_finish = {"Tram_2": 8 , "Tram_5": 5, "Tram_12": 7, "Tram_13": 8, "Tram_17": 9}

# Define a new function consisting of a while loop to calculate the number of passengers per hour per line
def min_passengers_per_hour(tram_data, min_passengers):
    results = []  # Initialize an empty list to store results
    for tram_name, i in tram_data.items():
        tram_passengers_per_hour_min = i * min_passengers
        print(f"{tram_name}: Minimum passengers per hour per direction = {tram_passengers_per_hour_min}")
        results.append(tram_passengers_per_hour_min)  # Append each result to the list
    return results

# Call the function and pass the dictionary of tram names and frequencies
min_results_finish = min_passengers_per_hour(tram_data_finish, passengers_tram_min)

# Calculate the sum of the minimum passenger results
total_min_passengers_finish = sum(min_results_finish)
print(f"Total minimum passengers per hour for tram: {total_min_passengers_finish}")

# Calculate the total for the situation with the largest trams, multiply by two to account for the two directions the trams and busses go in
print((2*total_min_passengers), "people can be transported per hour at minimum")

# Now define a similar function for the maximum number of passangers per hour per line
def max_passengers_per_hour(tram_data, max_passengers):
    results = []  # Initialize an empty list to store results
    for tram_name, i in tram_data.items():
        tram_passengers_per_hour_max = i * max_passengers
        print(f"{tram_name}: Maximum passengers per hour per direction = {tram_passengers_per_hour_max}")
        results.append(tram_passengers_per_hour_max)  # Append each result to the list
    return results

# Call the function and pass the dictionary of tram names and frequencies
max_results_finish = max_passengers_per_hour(tram_data_finish, passengers_tram_max)

# Calculate the sum of the minimum passenger results
total_max_passengers_finish = sum(max_results_finish)
print(f"Total maximum passengers per hour for tram: {total_max_passengers_finish}")

# Calculate the total for the situation with the largest trams, multiply by two to account for the two directions the trams and busses go in
print((2*total_max_passengers_finish), "people can be transported per hour at maximum")

Tram_2: Minimum passengers per hour per direction = 1024
Tram_5: Minimum passengers per hour per direction = 640
Tram_12: Minimum passengers per hour per direction = 896
Tram_13: Minimum passengers per hour per direction = 1024
Tram_17: Minimum passengers per hour per direction = 1152
Total minimum passengers per hour for tram: 4736
9472 people can be transported per hour at minimum
Tram_2: Maximum passengers per hour per direction = 1400
Tram_5: Maximum passengers per hour per direction = 875
Tram_12: Maximum passengers per hour per direction = 1225
Tram_13: Maximum passengers per hour per direction = 1400
Tram_17: Maximum passengers per hour per direction = 1575
Total maximum passengers per hour for tram: 6475
12950 people can be transported per hour at maximum


Once more by combining the data on the maximum number of passengers for a full bus and full trams (keeping in mind the the range of maximum passengers for the smallest and largest tram sizes) we get the results as shown above.

To summarize:
Between 9472 and 12950 people can be transported per hour depending on the variable size of the trams.

In [9]:
# Create a Folium map centered at your starting point
m2 = folium.Map(location = [x_coor2, y_coor2], zoom_start = 15)

# Add marker for the end point/finish of the swimming route
folium.Marker(location=[x_coor2, y_coor2], popup="End Point").add_to(m2)

# Add markers for the closest stops
for index, row in closest_stops.iterrows():
    coords = [row['geometry'].y, row['geometry'].x]
    
    # Create a custom icon to match the CircleMarker appearance
    custom_icon = folium.DivIcon(
        html=f'<div style="background-color: red; border-radius: 50%; width: 20px; height: 20px;"></div>',
        icon_size=(10, 10))
    folium.Marker(location=coords, icon=custom_icon).add_to(m2)

# Print the map with the locations
m2

As there are no relevant buslines servicing the nearby bus stops, only the nearest tram stops have been shown in the above map.